![mercado-libre](https://ml-challenge.mercadolibre.com/static/images/logo-mercado-libre_en.png)

# MeLi Data Challenge 2021
This notebook is my attempt on the 2021 Mercado Libre Data Challenge

## The Challenge
Build a model to forecast item inventory days based on Mercado Libre historical data.

## The Task
The task is to predict how long it will take for the inventory of a certain item to be sold completely. In inventory management theory this concept is known as inventory days.

In the evaluation set I will be given the item target stock, and I will have to provide a prediction for the number of days it will take to run out. Possible values range from 1 to 30. Rather than giving a point estimate, you are expected to provide a score for each the possible outcomes.

To put it simply, you need to answer the following question:

**'What are the odds that the target stock will be sold out in one day?', 'What about in two days?' and so on until day 30.**

## Repository
This notebook is hosted on the this repository: [github.com/matheusccouto/meli-data-challenge-2021](https://github.com/matheusccouto/meli-data-challenge-2021)

Check-out the different branches to see all approaches tested.

## Load Data

### Download data
Download challenge data from [ml-challenge.mercadolibre.com/downloads](https://ml-challenge.mercadolibre.com/downloads).

In [ ]:
import os

import requests

# Folder where I will place data.
DATA_DIR = "data"

# URL to download data.
DATA_URL_LIST = [
    r"https://meli-data-challenge.s3.amazonaws.com/2021/test_data.csv",
    r"https://meli-data-challenge.s3.amazonaws.com/2021/train_data.parquet",
    r"https://meli-data-challenge.s3.amazonaws.com/2021/items_static_metadata_full.jl",
    r"https://meli-data-challenge.s3.amazonaws.com/2021/sample_submission.csv.gz",
]


def download(url, path, ignore_if_exists=True):
    """Download files from the web."""
    # Check if it should be skipped.
    if ignore_if_exists and os.path.exists(path):
        return path  # return path to the user.
    # Request file.
    response = requests.get(url, allow_redirects=True)
    # Make sure the target folder exists.
    os.makedirs(os.path.dirname(path), exist_ok=True)
    # Save file.
    with open(path, mode="wb") as file:
        file.write(response.content)
    return path  # return path to the user.


for data_url in DATA_URL_LIST:
    # Create path for the file.
    data_name = os.path.basename(data_url)
    data_path = os.path.join(DATA_DIR, data_name)
    # Download file.
    print(f"Downloading {data_url} to {data_path}")
    download(url=data_url, path=data_path, ignore_if_exists=True)
    # Make sure download was succesful.
    assert os.path.exists(data_path)
    # And make sure the file is not damaged (at least 1MB)
    assert os.path.getsize(data_path) > 1e6

### Train Set
This dataset comprises two months of daily sales data for a subset of Mercadolibre SKUs (stock keeping units) . Each row corresponds to a particular date-SKU combination. Besides SKU and date, for each row, the following fields are available:

|Attributes|Description|
|---|---|
|sold_quantity|Number of units of the corresponding SKU that were sold on that particular date.|
|current_price|Point in time correct listing price.|
|currency|Currency in which the price is expressed.|
|listing_type|Type of listing the SKU had for that particular date. Possible values are classic or premium and they relate to the exposure the items receive and the fee charged to the seller as a sales comission.|
|shipping_logistic_type|Type of shipping method the SKU offered, for that particular date. Possible values are fulfillment, cross_docking and drop_off.|
|shipping_payment|Whether the shipping for the offered SKU at that particular date was free or paid, from the buyer's perspective.|
|minutes_active|Number of minutes the SKU was available for purchase on that particular date.|

In [ ]:
import pandas as pd

train_set = pd.read_parquet(os.path.join("data", "train_data.parquet"))
print(f"shape = {train_set.shape}")
train_set.head()

shape = (37660279, 9)


,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.0


### Test Set
For testing, the following file is provided test_data.csv. This file contains only two columns:

|Attribute|Description|
|---|---|
|SKU|indicates the SKU for which you have to make your prediction|
|target_stock|inventory level (aka number of units of the corresponding SKU for which you have to provide your estimation of inventory days.|

In [ ]:
test_set = pd.read_csv(os.path.join("data", "test_data.csv"))
print(f"shape = {test_set.shape}")
test_set.head()

shape = (551472, 2)


,sku,target_stock
0,464801,3
1,645793,4
2,99516,8
3,538100,8
4,557191,10


### Items Data
In the file items_static_metadata.jl there is some extra data related to the SKUs characteristics. The file contains a list of dictionaries where each of them contains metadata for a specific SKU . The following fields are available:

|Attribute|Description|
|---|---|
|SKU|stock-keeping-unit. This is a unique identifier for each distinct, physical inventory unit.|
|item_id|unique identifier of the listing the SKU belongs to. The same listing can be associated with more than one SKUs, for example, if different variations of the same item are offered in the listing.|
|item_domain_id|listing's domain id. A domain is a kind of listings clustering within MercadoLibre.|
|item_title|the listing's title in the marketplace.|
|site_id|the MercadoLibre's site the listing belongs to. The labels MLB, MLA and MLM refer to Brazil, Argentina and Mexico respectively.|
|product_id|listing product id. Field might be null for some listings.|
|product_id_family|listing product family id. Field might be null for some listings.|

In [ ]:
items_data = pd.read_json(os.path.join("data", "items_static_metadata_full.jl"), lines=True)
print(f"shape = {items_data.shape}")
items_data.head()

shape = (660916, 7)


,item_domain_id,item_id,item_title,site_id,sku,product_id,product_family_id
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,MLB,0,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB,1,None,None
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,MLM,2,None,None
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,MLM,3,None,None
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,MLB,4,None,None


### Sample Submission
A sample submission file for you to visualize the expected format of a submission.

In [ ]:
sample_submission = pd.read_csv(os.path.join("data", "sample_submission.csv.gz"), header=None)
print(f"shape = {sample_submission.shape}")
sample_submission.columns = range(1, 31, 1)  # Rename to match days of the month.
sample_submission.head()

shape = (551472, 30)


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,0.052,0.006,0.044,0.001,0.061,0.043,0.061,0.035,0.002,0.057,0.004,0.005,0.013,0.048,0.031,0.039,0.019,0.014,0.031,0.063,0.025,0.032,0.043,0.069,0.011,0.058,0.051,0.010,0.004,0.068
1,0.026,0.052,0.008,0.036,0.027,0.029,0.057,0.046,0.005,0.009,0.042,0.052,0.039,0.057,0.029,0.051,0.058,0.033,0.015,0.053,0.013,0.024,0.036,0.033,0.021,0.030,0.023,0.020,0.024,0.050
2,0.067,0.008,0.043,0.020,0.012,0.067,0.010,0.060,0.020,0.061,0.059,0.009,0.025,0.070,0.019,0.004,0.005,0.066,0.017,0.007,0.033,0.014,0.016,0.017,0.040,0.059,0.040,0.014,0.066,0.052
3,0.017,0.045,0.027,0.045,0.036,0.025,0.068,0.067,0.002,0.015,0.040,0.044,0.002,0.029,0.020,0.001,0.023,0.037,0.031,0.043,0.060,0.053,0.027,0.021,0.050,0.045,0.060,0.063,0.004,0.003
4,0.011,0.038,0.020,0.000,0.067,0.023,0.006,0.021,0.058,0.023,0.006,0.054,0.039,0.013,0.061,0.055,0.040,0.031,0.037,0.034,0.002,0.027,0.062,0.045,0.044,0.032,0.048,0.035,0.026,0.043


## Data Wrangling
In this section I will make sure the data is ready to be analyzed.

### Train Set

#### Dtypes
Make sure data is the appropiated format.

In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37660279 entries, 0 to 37660278
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   sku                     int64  
 1   date                    object 
 2   sold_quantity           int64  
 3   current_price           float64
 4   currency                object 
 5   listing_type            object 
 6   shipping_logistic_type  object 
 7   shipping_payment        object 
 8   minutes_active          float64
dtypes: float64(2), int64(2), object(5)
memory usage: 2.5+ GB


Date is stored as object and should be interpreted as date.

In [ ]:
train_set["date"] = pd.to_datetime(train_set["date"])
print(f"Dates go from {train_set['date'].min().strftime('%Y-%m-%d')} to {train_set['date'].max().strftime('%Y-%m-%d')}")

Dates go from 2021-02-01 to 2021-03-31


I'll also see the possible values from all other object columns to see if there's something to be done.

In [ ]:
for col in train_set.select_dtypes("object").columns:
    print(f"{col} classes are: {', '.join(train_set[col].unique().tolist())}")

currency classes are: REA, MEX, DOL, ARG
listing_type classes are: classic, premium
shipping_logistic_type classes are: fulfillment, cross_docking, drop_off
shipping_payment classes are: free_shipping, paid_shipping


#### Encoding
Object columns need to be encoded in order to be used in modelling. For the columns with only two classes I'll transform them into a single columns. The others with 3+ classes will receive a column for each column.

I won't drop one of them by now. Keeping all dummies may be bad for some algorithms, but for other dropping one of them is worse. I will keep all columns by now and decide what to do with these columns during modelling.

In [ ]:
train_set = pd.get_dummies(train_set, columns=["listing_type", "shipping_payment"], drop_first=True)
train_set = pd.get_dummies(train_set, columns=["currency", "shipping_logistic_type"], drop_first=False)
train_set.head()

,sku,date,sold_quantity,current_price,minutes_active,listing_type_premium,shipping_payment_paid_shipping,currency_ARG,currency_DOL,currency_MEX,currency_REA,shipping_logistic_type_cross_docking,shipping_logistic_type_drop_off,shipping_logistic_type_fulfillment
0,464801,2021-02-01,0,156.78,1440.0,0,0,0,0,0,1,0,0,1
1,464801,2021-02-02,0,156.78,1440.0,0,0,0,0,0,1,0,0,1
2,464801,2021-02-03,0,156.78,1440.0,0,0,0,0,0,1,0,0,1
3,464801,2021-02-04,0,156.78,1440.0,0,0,0,0,0,1,0,0,1
4,464801,2021-02-05,1,156.78,1440.0,0,0,0,0,0,1,0,0,1


#### NaN
Check for NaNs in the dataset.

In [ ]:
print(f"There are {train_set.isna().sum().sum()} NaNs in the dataset.")

There are 0 NaNs in the dataset.


### Test Set

#### Dtypes
Make sure data is the appropiated format.

In [ ]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551472 entries, 0 to 551471
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   sku           551472 non-null  int64
 1   target_stock  551472 non-null  int64
dtypes: int64(2)
memory usage: 8.4 MB


It also confirmed that there is no NaNs.

#### Sparse Matrix
If you notice the submission is a dense matrix with the probabilities of the item running out on that given day.

The be comparable, the test set should be a sparse binary matrix with the day the item actually ran out.

As the train set is way too big I will extract only the relevant columns and keep only day where at leat one item was sold.

In [ ]:
# TODO: Continue from here. Cum sum is not working and I do not know why.

train_set_small = train_set[["sku", "date", "sold_quantity"]]
train_set_small = train_set_small[train_set_small["sold_quantity"] > 0]
train_set_small = train_set_small.sort_values(["sku", "date"])
cum_qty_by_date = train_set_small.groupby(["sku", "date"]).sum()

del train_set_small

cum_qty_by_date.head()

sold_quantity
sku date                     
0   2021-03-17              1
    2021-03-22              2
    2021-03-23              1
    2021-03-24              4
    2021-03-25              2

### Items Data

#### Dtypes
Make sure data is the appropiated format.

In [ ]:
items_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 660916 entries, 0 to 660915
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   item_domain_id     660913 non-null  object
 1   item_id            660916 non-null  int64 
 2   item_title         660916 non-null  object
 3   site_id            660916 non-null  object
 4   sku                660916 non-null  int64 
 5   product_id         29668 non-null   object
 6   product_family_id  83184 non-null   object
dtypes: int64(2), object(5)
memory usage: 35.3+ MB


The first thing I notice here is that the `sku` should be this dataset index.

In [ ]:
items_data = items_data.set_index("sku")
items_data.head()

,item_domain_id,item_id,item_title,site_id,product_id,product_family_id
sku,,,,,,
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,MLB,None,MLB15832732
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,MLB,None,None
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,MLM,None,None
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,MLM,None,None
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,MLB,None,None


#### Encoding
Let's check if some of these columns are able to be encoded.

In [ ]:
for col in items_data.select_dtypes("object").columns:
    print(f"{col} has {len(items_data[col].unique())} classes")

item_domain_id has 8409 classes
item_title has 478175 classes
site_id has 3 classes
product_id has 15864 classes
product_family_id has 29601 classes


It seems only `site_id` is categorical. The remainded cannot be encoded by now.

For the same reason as before. I won't drop one of the encoded categories. I will leave this decision to later on.

In [ ]:
items_data = pd.get_dummies(items_data, columns=["site_id"], drop_first=False)
items_data.head()

,item_domain_id,item_id,item_title,product_id,product_family_id,site_id_MLA,site_id_MLB,site_id_MLM
sku,,,,,,,,
0,MLB-SNEAKERS,492155,Tênis Masculino Olympikus Cyber Barato Promoçao,None,MLB15832732,0,1,0
1,MLB-SURFBOARD_RACKS,300279,Suporte Rack Prancha Parede C/ Regulagem Horiz...,None,None,0,1,0
2,MLM-NECKLACES,69847,5 Collares Plateados Dama Gargantilla Choker -...,None,None,0,0,1
3,MLM-RINGS,298603,Lindo Anillo De Bella Crepusculo Twilight Prom...,None,None,0,0,1
4,MLB-WEBCAMS,345949,Webcam Com Microfone Hd 720p Knup Youtube Pc V...,None,None,0,1,0


#### NaNs
The columns `product_id` and `product_family_id` are mostly NaNs. I will deal with them later. 

The `item_domain_id` is has only a few missing rows. I will deal with them manually.

In [ ]:
items_data[items_data['item_domain_id'].isna()]

,item_domain_id,item_id,item_title,product_id,product_family_id,site_id_MLA,site_id_MLB,site_id_MLM
sku,,,,,,,,
454273,None,245417,Fone De Ouvido Multilaser Tws Ph326 Airbud Sem...,None,MLB16117735,0,1,0
459892,None,283198,Headset Gamer Multilaser P2 Ph123 Preto E Verde,None,MLB16158968,0,1,0
553503,None,114028,Prancha De Cabelo Taiff Elegance Frizz Cerâmic...,None,MLB17821818,0,1,0


In [ ]:
# Iterate over SKU that has missing data on item_domain_id.
for sku in items_data[items_data["item_domain_id"].isna()].index:
    print(f"sku = {sku}")
    print(f"item_title = {items_data.loc[sku]['item_title']}")

    # Filter only lines from same product_family_id.
    product_family_id = items_data[items_data["product_family_id"] == items_data.loc[sku]["product_family_id"]]

    # If the only line is the one that we are trying to fill, send a error message.
    if product_family_id["item_domain_id"].isna().all():
        print("No other products on the same family to compare to.\n")
    
    # But if it has, fill the missing values.
    else:
        item_domain_id = product_family_id["item_domain_id"].dropna().iloc[0]
        print(f"item_domain_id = {item_domain_id}\n")
        items_data.loc[sku, "item_domain_id"] = item_domain_id

sku = 454273
item_title = Fone De Ouvido Multilaser Tws Ph326 Airbud Sem Fio Microfone
item_domain_id = MLB-HEADPHONES

sku = 459892
item_title = Headset Gamer Multilaser P2 Ph123 Preto E Verde
No other products on the same family to compare to.

sku = 553503
item_title = Prancha De Cabelo Taiff Elegance Frizz Cerâmica 230ºc Bivolt
item_domain_id = MLB-HAIR_STRAIGHTENERS



The *Headset Gamer Multilaser P2 Ph123 Preto E Verde* must be filled manually. I'll fill it with *MLB-HEADPHONES*

In [ ]:
items_data.loc[459892, "item_domain_id"] = "MLB-HEADPHONES"
print(f"Amount of missing data = {items_data['item_domain_id'].isna().sum()}")

Amount of missing data = 0


## Data Analysis

## Feature Engineering

## Model Development

## Model Evaluation

### Metric
Submissions will be scored using the **Ranked Probability Score (RPS)** metric.

**Ranked Probability Score** is a squared measure that compares the **estimated cumulative density function** of a probabilistic forecast with the **actual cumulative density function** of the corresponding observation. Under a discrete scenario of possible outcomes, the RPS formula is as follows:

$$ RPS = \frac{1}{N}\sum_{i=1}^{N}\left[\sum_{K=1}^{K}(Y_K(i) - O_K(i))^2\right] $$

where:

* $N$ is the number of rows in the dataset,
* $O$ represents the target,
* $K$ is the number of classes, and
* $Y$ is the predicted probability.

The **RPS** metric can be more easily understood graphically. The figure below illustrates how the value **RPS** takes is simply the square of the area where the predicted cumulative probability function and the ground truth do not overlap each other.

![rps-intuition](https://ml-challenge.mercadolibre.com/static/images/2021/metric_fig_5.png)

The main advantage of **RPS**, in comparison with other common metrics for forecasting problems, is that it is **sensitive to distance**, which means that a forecast is increasingly penalized the more its predictions differ from the actual outcome. This property is especially attractive for the task at hand from a business perspective: in order to manage inventory efficiently at MercadoLibre’s fulfillment centers, we would like the forecast to be as close as possible to the actual value of inventory days, besides the fact of just hitting the target number of inventory days with the highest point-probability estimate.

Disclaimer: you do NOT have to submit CUMULATIVE PROBABILITIES. **You are expected to provide point-probability estimates and the scorer itself will compute the accumulation.**

In [ ]:
import numpy as np


def rps(y, o):
    """Ranked Probability Score"""
    if y_pred.shape != y_true.shape:
        raise ValueError(f"y shape is {y.shape}. Expected shape was {o.shape}")
    n = y.shape[0]
    k = y.shape[1]

    # Y is PDF and must be converted to CDF.
    y_cdf = np.cumsum(y)

    # TODO: Continue from here
